In [ ]:
# for imports
import random 

: 

In [ ]:
class Store:
    def __init__(self, store_id, store_name, branch, estimated_bags, rating, longitude, latitude):
        # private attributes
        self._store_id = store_id
        self._store_name = store_name
        self._branch = branch 
        self._rating = rating 
        self._price_per_bag = None 
        self._estimated_bags = estimated_bags # average_bags_at_9AM
        self._actual_bags = None # actual bags at night
        self._bags_reserved = 0 
        self._bags_canceled = 0 
        self._bags_sold = 0 
        self._longitude = longitude
        self._latitude = latitude

        # WHAT WE MIGHT ADD LATER:
        # cancellation rate history
        # location

    # setters
    def set_price_per_bag(self, price):
        self._price_per_bag = price 

    def set_actual_bags(self, count):
        self._actual_bags = count 


    # getters
    def get_store_id(self):
        return self._store_id
    
    def get_store_name(self):
        return self._store_name
    
    def get_branch(self):
        return self._branch
    
    def get_price_per_bag(self):
        return self._price_per_bag 

    def get_estimated_bags(self):
        return self._estimated_bags 

    def get_actual_bags(self):
        return self._actual_bags 
    
    def get_unreserved_bags(self):
        return self._estimated_bags - self._bags_reserved
    
    def get_rating(self):
        return self._rating
    
    def get_location(self):
        return {self._longitude, self._latitude}

    
    def reserve_bag(self):
        self._bags_reserved += 1

    def cancel_bag(self):
        self._bags_canceled += 1

    def sell_bag(self):
        self._bags_sold += 1


In [ ]:
class Customer:
    def __init__(self, customer_id, valuations, longitude, latitude):
        self._customer_id = customer_id
        self._valuations = valuations
        self._longitude = longitude
        self._latitude = latitude
        self._reserved_store_ID = None # store the store at which the customer reserved a bag from for future purposes (and printing purposes too)


    def get_customer_id(self):
        return self._customer_id
    
    def get_valuation(self, store_id):
        return self._valuations[store_id]
    
    def get_location(self):
        return {self._longitude, self._latitude}
    
    def get_reserved_store(self):
        return self._reserved_store_ID
    
    def reserve(self, store_id):
        self._reserved_store_ID = store_id


    

In [ ]:
class Marketplace:
    def __init__(self, n = 10, stores = None, customers = None): #either initialize with existing lists or add them later
        self._stores = []
        self._customers = [] 

        # default n is 10 for now, we can change the default later
        self._n = n 

        self.algorithm1 = Algorithm1()

    
    def add_store(self, store):
        self._stores.append(store)

    def add_customer(self, customer):
        self._customers.append(customer)

    def get_stores(self):
        return self._stores
    
    def get_customers(self):
        return self._customers
    

    def select(self, customer): 
        """
        This is the main algorithm that generates the n stores to display.

        Inputs: 
            customer -> customer we are choosing for

        Output: a list of n Store objects
        """
        # for the purposes of milestone 1 simulation, return all stores
        return self._stores

        #return self._greedy_baseline()
        # return algorithm1.select(self._stores, self._n, customer)


    def _greedy_baseline(self): 
      # select just the top-n popular stores (based on ranking ONLY)
      return None

    
        

In [ ]:
class Algorithm1:
    def __init__(self):
        pass


    def select(self, stores, n, customer):
        selected_stores = []
        # First, check the stores that the customer rated 3+ that still have unreserved bags today
        liked_stores = self._get_high_valuation_stores(self, stores, customer) 
        
        selected_stores += liked_stores

        # IN ALL THE FUNCTIONS, WE ONLY CONSIDER STORES THAT HAVE UNRESERVED BAGS 


        # if number of liked stores is exactly n, then we just return those (selection function already sorts them)
        if len(liked_stores) == n:
             return selected_stores
        
        # if number of liked stores = 0 (for example, none of the stores they liked still has unreserved or its their first time using the app)
        # we use the main selection function (we will call it the general function) to select based on store rating, price, location and thats it
        if len(liked_stores) == 0:
            selected_stores = self._select_general(self, n, stores, customer)
            return selected_stores
            

        # if the number of liked stores < n, we also use the general function to fill the rest of n
        # we get x = (n - number of liked stores)
        # we select x those based on store rating, price, location
        if len(liked_stores) < n:
            x = self._n - len(liked_stores)
            selected_stores += self._select_general(self, x, stores, customer)
            return selected_stores
        

        # final scenario: if number of liked stores exceeds n, we use a function that takes into consideration the customer's valuation
        # we will call this function the valued selection. It is different than the general function in that it additionally uses valuation 
        # (as well as store rating, price, location)
        # for now, we will only consider the subarray of stores they liked
        # FLAW HERE!!! in this case user will not get the chance to try new stores
        if len(liked_stores) > n:
            selected_stores = self._select_valued(self, n, customer, liked_stores)
            return selected_stores


    def _get_high_valuation_stores(self, stores, customer):
            # first we have an empty array
            # for each store in self._stores
                # if store.get_unreserved_bags() > 0
                # if customer.get_valuation(store) >= 3
                    # add store to array

            # sort those stores based on either price, location, or valuation (need to decide on this)
            # return array
            pass


    def _select_general(self, x, stores, customer):
        # Implement an algorithm using these factors:
            # store.get_rating() for each store in self._stores array
            # store.get_price_per_bag() for each store in self._stores array
            # store.get_location() for each store in self._stores array (returns {longitude, latitude})

        # dont forget to first check if a store has store.get_unreserved_bags() > 0 before considering it
        pass 
    

In [ ]:
class Algorithm2: # menna's strategy
    pass


In [ ]:
class Algorithm3: # nadine's strategy
    pass

In [ ]:
class Algorithm4: # ziad's strategy
    pass

In [ ]:
def main():
    # MAIN FUNCTION!!!!!!!!!!!!!!
    # here is where we will simulate
    basic_simulation() 


def basic_simulation(): # basic simulation for milestone 1
    # disclaimer: sample stores and customers generated by AI

    # generate random stores and customers
    # return all of the stores for each customer
    app = Marketplace()

    # Create sample stores
    store1 = Store(store_id = 1, store_name = "Bakery A", branch = "Downtown",
                   estimated_bags = 10, rating = 4.5, longitude = 30.1, latitude = 31.2)

    store2 = Store(store_id = 2, store_name = "Bakery B", branch = "Mall",
                   estimated_bags = 7, rating = 4.0, longitude = 30.2, latitude = 31.3)

    store3 = Store(store_id = 3, store_name = "Bakery C", branch = "Station",
                   estimated_bags = 12, rating = 4.9, longitude = 30.3, latitude = 31.4)

    # add stores to app
    app.add_store(store1)
    app.add_store(store2)
    app.add_store(store3)

    # Create customers
    customer1 = Customer(customer_id = 101, valuations = {1:5, 2:3, 3:4}, longitude = 30.0, latitude = 31.0)
    customer2 = Customer(customer_id = 102, valuations = {1:2, 2:5, 3:1}, longitude = 30.15, latitude = 31.05)

    app.add_customer(customer1)
    app.add_customer(customer2)

    # simulate
    print("=== BASIC SIMULATION START ===\n")

    for customer in app.get_customers():
        print(f"Customer {customer.get_customer_id()} views the following stores:")

        selected_stores = app.select(customer)

        for i, store in enumerate(selected_stores, start=1):
            print(f"{i}: {store.get_store_name()}")

        print()

    print("=== SIMULATION END ===")


if __name__ == "__main__":
    main()